In basic RAG pipeline LLMs are only used for synthesis. router shows hoq to use our LLM to make a decision by picking choice of different pipelines.

Here we will show you how to use LLM to not only pick a function to execute, but also infer an argument to pass through function.

One of the promises of LLms is their ability to take actions and interact with external environment and necessary component to make this possible is a good interface for the LLMs to use.


Using Tool calling, in basic RAG pipeline, LLMs are only used for synthesis of information only,

we can use LLMs to pick the best query pipeline to answer the user query. This is simplified form of tool calling.

Tool calling enables LLM to interact with the external environments through a dynamic interface where the tool calling not only helps choosing the appropriate tool but also infer necessary arguments for execution.

here we will sjow how to use an LLM to not only pick a function to execute, but also infer an argument to pass to function.

This allows LLM to how to use vectordb instead of just consuming its outputs. And the final results, is that users are able to ask more questions and get abck more precise results than standard RAG techniques through tool calling.

Tool calling adds a layer of query understanding on top of a RAG pipeline, enable user to ask complex queries and get more precise results.

In [1]:
import os 
from dotenv import load_dotenv


In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


In [3]:
import nest_asyncio
nest_asyncio.apply()

How to find a tool interface from a python function, and the LLM will automatically infer the parameters from the signature of the python functions using LlamaIndex abstractions.

In [ ]:
#calculator functions

# FunctionTool wraps any given Python Function that you feed it. And so we see that the function tool takes in both the add function defined here,as well as mystery function
# which is just (x+y) * (x+y) we can see both add and mystery functions have type annotations for both x and y variables, as well as the docstring.


from llama_index.core.tools import FunctionTool

def add(x:int, y:int) -> int:
    """Adds two integers together."""  # is used as prompt for the LLM
    return x+y

def mystery(x:int, y:int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x+y) * (x+y)

add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [6]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose = True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


we can say above example is the extended version of the router, not only the LLM pick the tool, but also decides what parameters to give to the tool.

Will define Agentic layer on top of vector search, now not only LLM choose vector search we can also get it to infer metadata filters, which is structured list of tags

that helps to return a more precise set of search results.

In [7]:
# load the documents

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files = ["/Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf"]).load_data()

In [8]:
# split the document into set of even chunks or nodes with a chunk size of 1024.
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [12]:
print(nodes[5].get_content(metadata_mode="all"))

page_label: 5
file_name: metagpt.pdf
file_path: /Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-05-31
last_modified_date: 2025-05-31

Preprint
Figure 3: A diagram showing the software development process in MetaGPT, emphasizing its sig-
nificant dependence on SOPs. The more detailed demonstration can be found in Appendix B.
Specifically, as shown in Figure 1, upon obtaining user requirements, the Product Manager under-
takes a thorough analysis, formulating a detailed PRD that includes User Stories and Requirement
Pool. This serves as a preliminary functional breakdown. The structured PRD is then passed to
the Architect, who translates the requirements into system design components, such as File Lists,
Data Structures, and Interface Definitions. Once captured in the system design, the information is
directed towards the Project Manager for task distribution. Engineers proceed to execute the des-
ignated cl

In [13]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

Calling RAG pipeline using metadata filters

In [15]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key":"page_label", "value":"2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?"
)

In [16]:
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with 85.9% and 87.7% in Pass@1, standing out in handling higher levels of software complexity, offering extensive functionality, and achieving a 100% task completion rate in experimental evaluations.


In [17]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '/Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-05-31', 'last_modified_date': '2025-05-31'}


# Enhancing Data Retrieval

1. Integrating Metadata Filters into a retrieval tool function. :- This function takes in both the query string and page numbers as filters.
    
    The LLM then actually infer the page numbers to filter for a user query, instead of actually having the user manually specify the metadata filters.

2. This function enables more precise retrieval by accepting a query string and optional metadata filters, such as page numbers.

3. The LLM can intelligently infer relevant metadata filters (eg. page numbers) based on the user's query.

4. You can define different type of metadata filters like section IDs, headers, or footers,...

In [19]:
from typing import List
from llama_index.core.vector_stores import FilterCondition

def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    query(str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
    over all pages. Otherwise, filter by the set of specified pages.
    
    """
    metadata_dicts = [
        {"key":"page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k = 2,
        filters = MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name = "vector_tool",
    fn = vector_query
)

In [21]:
llm = OpenAI(model="gpt-3.5-turbo",temperature=0)
response = llm.predict_and_call(
    [vector_query_tool],
    "what are the high-level results of MetaGPT as described on page 2?",
    verbose = True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with 85.9% and 87.7% in Pass@1. It outperforms other popular frameworks like AutoGPT, LangChain, AgentVerse, and ChatDev in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, MetaGPT demonstrates a 100% task completion rate, showcasing its robustness and efficiency in terms of time and token costs.


In [22]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '/Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-05-31', 'last_modified_date': '2025-05-31'}


In [23]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async = True
)
summary_tool = QueryEngineTool.from_defaults(
    name = "summary_tool",
    query_engine = summary_query_engine,
    description =(
        "Useful if you want to get a summary of MetaGPT"
    ),
)


In [25]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose = True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various aspects. For example, MetaGPT achieves a higher score in executability, takes less time for execution, requires more tokens but fewer tokens to generate one line of code compared to ChatDev. Additionally, MetaGPT also outperforms ChatDev in terms of code statistics and the cost of human revision.


In [26]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What is the summary of the paper?",
    verbose = True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "This paper discusses the impact of climate change on biodiversity and the environment."}
=== Function Output ===
I cannot provide an answer to the query as there is no information related to climate change, biodiversity, or the environment in the provided context.
